In [676]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import torch 

In [677]:
with open('../../sf_vs_nyc_house_data.csv', 'r') as f:
    labels = f.readline()

print(labels)

full_data = np.genfromtxt('../../sf_vs_nyc_house_data.csv', delimiter=',')[1:]
full_data.shape

in_sf,beds,bath,price,year_built,sqft,price_per_sqft,elevation



(492, 8)

In [678]:
X_train, X_test, y_train, y_test = train_test_split(full_data[:,1:], full_data[:,:1], test_size=.2)

print(f' Train shape X: {X_train.shape} y: {y_test.shape} \n  Test shape X: {X_test.shape}  y: {y_test.shape} ')

 Train shape X: (393, 7) y: (99, 1) 
  Test shape X: (99, 7)  y: (99, 1) 


---
NOW THAT WE HAVE THAT WE CAN START TO MAKE THE NN


In [679]:
# class NN_torch(torch.nn.Module):
    
#     def __init__(self):
#         super(NN_torch, self).__init__()

#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(7,1),
#             torch.nn.ReLU(),
#             torch.nn.Linear(1,1)
#         )


In [680]:
class NN():

    def __init__(self, input_size, hidden_sizes, out_size) -> None:
        self.l1 = np.random.rand(input_size,hidden_sizes[0])
        self.b1 = np.zeros((1,hidden_sizes[0]))
        self.l2 = np.random.rand(hidden_sizes[0],out_size)
        self.b2 = np.zeros((1,out_size))

    def activation_func(self, o, type):
        if type.lower()== 'relu':
            return np.where(o >= 0, o, 0)
        if type.lower() == 'softmax':
            #make val stable
            o_stable = o - np.max(o, axis=1, keepdims=True)
            o_exp = np.exp(o_stable) #e^x for each x in 0_stable
            return o_exp / np.sum(o_exp, axis=1, keepdims=True)
        if type.lower() == 'sigmoid':
            return 1/(1+np.exp(-o))
        
    def forward(self, X):
        self.X = X
        self.o1 = X @ self.l1 + self.b1
        self.h1 = self.activation_func(self.o1, 'relu')
        self.o2 = self.h1 @ self.l2 + self.b2
        self.y_hat = self.activation_func(self.o2, 'sigmoid')
        return self.y_hat
    
    def calc_loss(self, y_true, y_hat, type):
        if type.lower() == 'mse':
            return np.sum((y_true - y_hat) ** 2) / len(y_true)

    def grad_relu(self, idx=None):
        if idx is None:
            return (self.h1 > 0).astype(float) #the as type flaot turn the bool mask in 0 or 1 which is what we wnat for the relu derivative
        else:
            return (self.h1[idx] > 0).astype(float)

    def grad_mse(self, y_true): #return vextor (training_samps x 1)
        return 2 * (self.y_hat - y_true) / len(y_true)
    
    def grad_sigmoid(self): #return vextor (training_samps x 1)
        return self.y_hat * (1 - self.y_hat)

    def dl_wrt_q0(self, y_true): #return vextor (training_samps x size of layer 2)
        #print(f'this is grad wrt_q0 doe the sconf layetr: {(self.grad_mse(y_true) * self.grad_sigmoid() * self.o1).shape}')
        return self.grad_mse(y_true) * self.grad_sigmoid() * self.o1

    def dl_wrt_b2(self, y_true):
        return self.grad_mse(y_true) * self.grad_sigmoid()

    def dl_wrt_b1(self, y_true):
        # print((np.sum(self.grad_mse(y_true), axis=0)).shape)
        # print((np.sum(self.grad_sigmoid(), axis=0)).shape)
        # print(self.l2.shape)
        # print(np.sum((self.grad_relu()), axis=0).reshape(self.l2.shape).shape,'\n')
        
        return np.sum(self.grad_mse(y_true), axis=0) * np.sum(self.grad_sigmoid(), axis=0) * self.l2 * np.sum((self.grad_relu()), axis=0).reshape(self.l2.shape)

    def grad_wrt_w(self, y_true):
        #(1 x training_samps) is the dl_wrt_b1(y_true).T then we multiply accordily by the X whis is (training_samps, 7) to ge full chainrule
        #if we did dl_wrt_b1(y_true).T * one specific row we would get the gradiet for each weight in layer one for one training exmple 
        #so the do for all and them summ across the colomns or jut the dpr product
        print(f'{self.grad_mse(y_true).shape} ')
        print(self.grad_sigmoid().shape)
        print(self.l2.shape)
        print(self.grad_relu().shape)

        dl_wrt_wi = self.grad_mse(y_true) * self.grad_sigmoid() * self.l2.T * self.grad_relu()
        return np.dot(dl_wrt_wi.T, self.X)

    def backprop(self, alpha, y_true):
        # print(f'l1 before this : {self.l1},new gradit l1: {self.grad_wrt_w(y_true)}')
        # print(f'l2 before this : {self.l2},new gradit l2: {alpha * self.dl_wrt_q0(y_true)}')
        # print(f'b1 before this : {self.b1},new gradit l1: {alpha * self.dl_wrt_b1(y_true)}')
        # print(f'b2 before this : {self.b2},new gradit l1: {alpha * self.dl_wrt_b2(y_true)}\n\n')

        self.b2 -= alpha * np.sum(self.dl_wrt_b2(y_true), axis=0)
        self.l2 -= alpha * np.sum(self.dl_wrt_q0(y_true),axis=0).reshape(self.l2.shape)
        self.b1 = self.b1 - (alpha * self.dl_wrt_b1(y_true)).T
        #self.b1 -= alpha * self.dl_wrt_b1(y_true)
        self.l1 -= alpha * self.grad_wrt_w(y_true).reshape(self.l1.shape)




In [681]:
input_size = 7
hidden_sizes = [3]
out_size = 1

custom_nn = NN(input_size, hidden_sizes, out_size)

class NN_torch(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes, out_size):
        super(NN_torch, self).__init__()
        self.l1 = torch.nn.Linear(input_size, hidden_sizes[0])
        self.l2 = torch.nn.Linear(hidden_sizes[0], out_size)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, X):
        o1 = self.l1(X)
        h1 = self.relu(o1)
        o2 = self.l2(h1)
        y_hat = self.sigmoid(o2)
        return y_hat

In [682]:
model = NN_torch(input_size, hidden_sizes, out_size)
with torch.no_grad():
    model.l1.weight = torch.nn.Parameter(torch.tensor(custom_nn.l1.T, dtype=torch.float32))
    model.l1.bias = torch.nn.Parameter(torch.tensor(custom_nn.b1.flatten(), dtype=torch.float32))
    model.l2.weight = torch.nn.Parameter(torch.tensor(custom_nn.l2.T, dtype=torch.float32))
    model.l2.bias = torch.nn.Parameter(torch.tensor(custom_nn.b2.flatten(), dtype=torch.float32))

In [683]:
# Verify the parameters are set correctly
print("l1 :\n", custom_nn.l1)
print("torch l1:\n", model.l1.weight.data.numpy().T)

print("b1 :\n", custom_nn.b1)
print("torch b1:\n", model.l1.bias.data.numpy())

print("l2 :\n", custom_nn.l2)
print("torch l2 :\n", model.l2.weight.data.numpy().T)

l1 :
 [[0.78594153 0.29394232 0.21848912]
 [0.61006718 0.20106184 0.17582819]
 [0.48008921 0.27407688 0.56593221]
 [0.79211208 0.14628843 0.14534152]
 [0.56111222 0.56400843 0.62073934]
 [0.52517558 0.23783087 0.35430834]
 [0.61155796 0.78962477 0.71271374]]
torch l1:
 [[0.78594154 0.2939423  0.21848911]
 [0.6100672  0.20106184 0.17582819]
 [0.48008922 0.27407688 0.5659322 ]
 [0.79211205 0.14628844 0.14534152]
 [0.5611122  0.5640084  0.62073934]
 [0.5251756  0.23783088 0.35430834]
 [0.61155796 0.78962475 0.7127137 ]]
b1 :
 [[0. 0. 0.]]
torch b1:
 [0. 0. 0.]
l2 :
 [[0.30311333]
 [0.29498333]
 [0.36033728]]
torch l2 :
 [[0.30311334]
 [0.29498333]
 [0.3603373 ]]


In [684]:
logits_torch = model.forward(torch.Tensor(X_train))

In [685]:

logits_custom = custom_nn.forward(X_train)
#logits_custom

In [686]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [687]:

alpha = 0.001
custom_nn.backprop(alpha, np.array(y_train))


loss = criterion(logits_torch, torch.Tensor([y_train]))

# Perform backpropagation and update weights
optimizer.zero_grad()
loss.backward()
print("\n\n Gradients for troucbh nn:")
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name} grad:\n{param.grad}")


optimizer.step()


# Verify the parameters are set correctly
print("\nl1 :",custom_nn.l1)
print("b1 :", custom_nn.b1)
print("l2 :", custom_nn.l2)
print("b2 :", custom_nn.b2)



print('\n\n',"Updated parameters:")
for name, param in model.named_parameters():
    print(name, param.data)

(393, 1) 
(393, 1)
(3, 1)
(393, 3)


 Gradients for troucbh nn:
l1.weight grad:
tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])
l1.bias grad:
tensor([0., 0., 0.])
l2.weight grad:
tensor([[0., 0., 0.]])
l2.bias grad:
tensor([0.])

l1 : [[0.78594153 0.29394232 0.21848912]
 [0.61006718 0.20106184 0.17582819]
 [0.48008921 0.27407688 0.56593221]
 [0.79211208 0.14628843 0.14534152]
 [0.56111222 0.56400843 0.62073934]
 [0.52517558 0.23783087 0.35430834]
 [0.61155796 0.78962477 0.71271374]]
b1 : [[0. 0. 0.]]
l2 : [[0.30311333]
 [0.29498333]
 [0.36033728]]
b2 : [[0.]]


 Updated parameters:
l1.weight tensor([[0.7859, 0.6101, 0.4801, 0.7921, 0.5611, 0.5252, 0.6116],
        [0.2939, 0.2011, 0.2741, 0.1463, 0.5640, 0.2378, 0.7896],
        [0.2185, 0.1758, 0.5659, 0.1453, 0.6207, 0.3543, 0.7127]])
l1.bias tensor([0., 0., 0.])
l2.weight tensor([[0.3031, 0.2950, 0.3603]])
l2.bias tensor([0.])


/var/folders/g7/qrp04c4n56q06wny0phpjyrc0000gn/T/ipykernel_20268/503546361.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  loss = criterion(logits_torch, torch.Tensor([y_train]))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 393, 1])) that is different to the input size (torch.Size([393, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
